<a href="https://colab.research.google.com/github/srikannyaroy/DL-assignment/blob/main/mhc2024006_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5,))
])

train_data = datasets.MNIST(
    root = "./data",
    train = True,
    transform = transform,
    download = True
)

test_data = datasets.MNIST(
    root = "./data",
    train = False,
    transform = transform,
    download = True
)

train_loader = DataLoader(train_data, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 32, shuffle = True)

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.24MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 28.9k/28.9k [00:00<00:00, 157kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.46MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.51MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, num_classes=10):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [ ]:
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

In [ ]:
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

In [ ]:
train_model(model, train_loader, criterion, optimizer, epochs=5)
evaluate_model(model, test_loader)

Epoch [1/5], Loss: 0.0766
Epoch [2/5], Loss: 0.0333
Epoch [3/5], Loss: 0.0669
Epoch [4/5], Loss: 0.0574
Epoch [5/5], Loss: 0.1924
Accuracy: 95.43%


95.43

In [ ]:
architectures = [
    [784, 128, 10],
    [784, 256, 128, 10],
    [784, 512, 256, 128, 10]
]

learning_rates = [0.1, 0.01, 0.001]
activation_functions = [nn.ReLU(), nn.Sigmoid(), nn.Tanh()]

In [ ]:
for arch in architectures:
    for lr in learning_rates:
        for activation in activation_functions:
            print(f"Architecture: {arch}, Learning Rate: {lr}, Activation: {activation}")
            # Create a model dynamically
            class DynamicNN(nn.Module):
                def __init__(self, layers, activation_fn):
                    super(DynamicNN, self).__init__()
                    self.layers = nn.ModuleList()
                    for i in range(len(layers) - 1):
                        self.layers.append(nn.Linear(layers[i], layers[i + 1]))
                    self.activation = activation_fn

                def forward(self, x):
                    x = x.view(x.size(0), -1)
                    for layer in self.layers[:-1]:
                        x = self.activation(layer(x))
                    x = self.layers[-1](x)
                    return x

            model = DynamicNN(arch, activation).to(device)
            optimizer = optim.Adam(model.parameters(), lr=lr)
            train_model(model, train_loader, criterion, optimizer, epochs=5)
            accuracy = evaluate_model(model, test_loader)

Architecture: [784, 128, 10], Learning Rate: 0.1, Activation: ReLU()
Epoch [1/5], Loss: 2.2860
Epoch [2/5], Loss: 2.3281
Epoch [3/5], Loss: 2.3116
Epoch [4/5], Loss: 2.3869
Epoch [5/5], Loss: 2.3690
Accuracy: 9.58%
Architecture: [784, 128, 10], Learning Rate: 0.1, Activation: Sigmoid()
Epoch [1/5], Loss: 2.9936
Epoch [2/5], Loss: 2.6005
Epoch [3/5], Loss: 2.9911
Epoch [4/5], Loss: 3.0716
Epoch [5/5], Loss: 2.9619
Accuracy: 9.80%
Architecture: [784, 128, 10], Learning Rate: 0.1, Activation: Tanh()
Epoch [1/5], Loss: 3.4488
Epoch [2/5], Loss: 4.4400
Epoch [3/5], Loss: 3.7465
Epoch [4/5], Loss: 3.6199
Epoch [5/5], Loss: 3.1433
Accuracy: 10.28%
Architecture: [784, 128, 10], Learning Rate: 0.01, Activation: ReLU()
Epoch [1/5], Loss: 0.4105
Epoch [2/5], Loss: 0.3964
Epoch [3/5], Loss: 0.3106
Epoch [4/5], Loss: 0.3397
Epoch [5/5], Loss: 0.2088
Accuracy: 91.04%
Architecture: [784, 128, 10], Learning Rate: 0.01, Activation: Sigmoid()
Epoch [1/5], Loss: 0.5591
Epoch [2/5], Loss: 0.8857
Epoch [3/